In [1]:
import polars as pl
import pyproj

In [2]:
# Projection from WGS84 to Pseudo-Mercator
transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

def project_struct_lonlat(s):
    """Returns a list of lists"""
    return transformer.transform(s["lon"], s["lat"])

In [3]:
trips = (
    pl.read_parquet("data/biketrips/*.parquet")
    .with_columns(
        pl.struct(
            pl.col("lon_start").alias("lon"),
            pl.col("lat_start").alias("lat")
        )
        .map_elements(project_struct_lonlat).alias("lonlat")
    )
    .with_columns(
        pl.col("lonlat").list.get(0).alias("lon"),
        pl.col("lonlat").list.get(1).alias("lat"),
    )
    .select(
        pl.col("lon"),
        pl.col("lat"),
    )
)

/var/folders/3d/ldmd42_x7zx8d8xqy0fkhmgc0000gn/T/ipykernel_82341/457137960.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


In [4]:
trips.plot.points(x="lon", y="lat",
                  datashade=True,
                  cmap="fire",
                  tiles="CartoDark",
                  width=800, height=600,
                  tools=["fullscreen"])

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .RGB.I   :RGB   [lon,lat]   (R,G,B,A)